In [1]:
from IPython.display import display, Markdown
import os
from LS_AMG_RAG.data_snythesis import prompt_utils
import google.generativeai as palm

import chromadb
import chromadb.utils.embedding_functions as embedding_functions
# chroma_client = chromadb.Client()
chroma_client = chromadb.PersistentClient(path="./")
gemini = prompt_utils.Gemini()
google_ef  = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=os.environ['GEMINI_API_KEY'])

In [2]:
documents = []
metadata = []
ids = []
# for idx, file_name in enumerate(os.listdir("../data/business_docs")):
#     with open(f"../data/business_docs/{file_name}", "r") as f:
#         file_contents = f.read()
#         documents.append(file_contents)
#         metadata.append({
#             "type": "business document",
#         })
#         ids.append(f"business_doc_{idx}")

# add all files within the 'data/' directory to chromadb using os.walk
for root, dirs, files in os.walk("../data"):
    for file in files:
        if file.endswith(".md"):
            category = root.split('\\')[-1]
            with open(os.path.join(root, file), "r") as f:
                file_contents = f.read()
                documents.append(file_contents)
                metadata.append({
                    "type": category,
                })
                ids.append(f"{category}_{file}")

In [2]:
len(documents)

NameError: name 'documents' is not defined

In [4]:
for m in palm.list_models():
    print(m.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision
models/embedding-001
models/aqa


In [5]:
collection = chroma_client.create_collection(name="my_collection", embedding_function=google_ef)

In [6]:
collection.add(
    documents=documents,
    metadatas=metadata,
    ids=ids,
)

In [7]:
len(collection.get()['ids'])

74

In [8]:
queries = [
    "What is Instagram's current business proposal?",
]
import time
total_start_time = time.time()
start_time = time.time()
results = collection.query(
    query_texts=queries,
    n_results=3
)
end_time = time.time()

print(f"Document retrieved: {results['ids'][0][-1]}")

print(f"Retrieval time: {end_time - start_time} seconds")

metaprompt = """You are a helpful and informative bot that answers questions using text from the reference document included below. \
Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
strike a friendly and converstional tone. \
Use your own knowledge base in addition to the information provided in the document to answer the question. \
Make relevant assumptions and use your best judgement to answer the question. \
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

  ANSWER:
"""

start_time = time.time()
gemini_result = gemini.send_message(message=metaprompt.format(query=queries[0], relevant_passage=results['documents'][0][-1])).text
end_time = time.time()
total_end_time = time.time()
display(Markdown(gemini_result))
print(f"Gemini time: {end_time - start_time} seconds")
print(f"Total time: {total_end_time - total_start_time} seconds")

Document retrieved: company_bylaws_About Instagram.md
Retrieval time: 0.46810412406921387 seconds


Instagram's business proposal is based on connecting users with brands, businesses, and one another. The platform focuses on providing a user-friendly experience and continuously iterating its features to meet the needs of its community. Instagram also prioritizes user feedback, implements strict content policies, and takes a proactive approach to moderation to ensure a safe and inclusive environment.

Gemini time: 2.662595272064209 seconds
Total time: 3.130699396133423 seconds


In [9]:
metaprompt = """You are a helpful and informative bot that answers questions using text from the reference passage included below. \
  Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
  However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
  strike a friendly and converstional tone. \
  If the passage is irrelevant to the answer, you may ignore it.
  QUESTION: '{query}'
  PASSAGE: '{relevant_passage}'

  ANSWER:
"""

start_time = time.time()
gemini_result = gemini.send_message(message=metaprompt.format(query=queries[0], relevant_passage=results['documents'][0][-1])).text
end_time = time.time()
display(Markdown(gemini_result))
print(f"Query time: {end_time - start_time} seconds")

Instagram's current business proposal is to connect users with brands, businesses, and each other. The platform focuses on providing a user-friendly experience and continuously iterating its features to meet the needs of its community. Instagram also prioritizes user feedback, implements strict content policies, and takes a proactive approach to moderation to ensure a safe and inclusive environment.

Query time: 2.6425905227661133 seconds


In [3]:
questions = [
"What is Instagram's current business proposal?",
"Can you provide details about the marketing plan for Instagram?",
"What information does the progress report of Instagram contain?",
"Who are the members of Instagram's board of directors?",
"Could you provide specific details from Instagram's employee contracts?",
"Can you share the latest balance sheet figures for Instagram?",
"What are the diversity and inclusion initiatives implemented by Instagram?",
"How do the terms of the employment contract boost employee retention?",
"What is the Marketing Objective for Influencer Collaboration Services?",
"Who is the target audience of Content Creation and Curation Services?",
"What is the financial update for the Reels Optimization Project?",
"Give me details about the progress report of the Stories Upgrade project.",
"Compare the progress report of the Feed Redesign and Stories Upgrade project and draw a conclusion on the information.",
"Who are the key personnel of Instagram?",
"List all the achievements of Instagram.",
"How does Instagram promote community involvement?",
"What is the background of the CEO (Chief Executive Officer) of Instagram?",
"Who is the chairman of Instagram and which year did the person become the chairman of the board?",
"How has the Board of Directors contributed to Instagram's success?",
"Does Instagram have any initiatives for Employee Resource Groups? If yes explain the initiative.",
"How does Instagram promote diversity?",
"Describe the ownership structure of Instagram.",
"What is the total amount of assets that Instagram has?",
"How much cash is present in balance with Instagram?",
"How much amount of long-term loan does Instagram have on the balance sheet?",
"What is the net income of Instagram?",
"What is the gross profit of Instagram?",
"What is the income before taxes Instagram and net income? Also let me know how much income tax is paid by Instagram.",
"What is the investment income of Instagram?",
"How much of Instagram income goes to Charitable Contributions?",
"What is the Position of Ava Thomas and in which department does she work?",
"Total number of work hours of Leo Kim?",
"What are the employee benefits provided to Ethan Rodriguez?",
"Who is the Quality Assurance Specialist at Instagram?",
"Name the employee working in the Security Department and what is his/her compensation?",
"What is the responsibility of the content moderator?",
"Which employee is getting paid $125,000 per year as compensation and which department does he work in?",
"Which employee did enter into the ESOP with Instagram on April 1, 2022?",
"How many number of shares were allocated to Liam Kim?",
"What is the Basic salary of Jane Smith?",
"How much bonus does the Public Relations Manager receive?",
"What is the net pay to the employee working in the Customer Support department?"
]

In [4]:
questions

["What is Instagram's current business proposal?",
 'Can you provide details about the marketing plan for Instagram?',
 'What information does the progress report of Instagram contain?',
 "Who are the members of Instagram's board of directors?",
 "Could you provide specific details from Instagram's employee contracts?",
 'Can you share the latest balance sheet figures for Instagram?',
 'What are the diversity and inclusion initiatives implemented by Instagram?',
 'How do the terms of the employment contract boost employee retention?',
 'What is the Marketing Objective for Influencer Collaboration Services?',
 'Who is the target audience of Content Creation and Curation Services?',
 'What is the financial update for the Reels Optimization Project?',
 'Give me details about the progress report of the Stories Upgrade project.',
 'Compare the progress report of the Feed Redesign and Stories Upgrade project and draw a conclusion on the information.',
 'Who are the key personnel of Instagram

In [2]:
collection = chroma_client.get_or_create_collection(name="my_collection", embedding_function=google_ef)

In [3]:
import time
def generate_responses(quer,collection):
    total_start_time = time.time()
    start_time = time.time()
    results = collection.query(
        query_texts=quer,
        n_results=3
    )
    end_time = time.time()

    print(f"Document retrieved: {results['ids'][0][0]}")

    print(f"Retrieval time: {end_time - start_time} seconds")

    metaprompt = """You are a helpful and informative bot that answers questions using text from the reference document included below. \
    Be sure to respond in a complete sentence, being comprehensive, including all relevant background information. \
    However, you are talking to a non-technical audience, so be sure to break down complicated concepts and \
    strike a friendly and converstional tone. \
    Use your own knowledge base in addition to the information provided in the document to answer the question. \
    Make relevant assumptions and use your best judgement to answer the question. \
    QUESTION: '{query}'
    PASSAGE: '{relevant_passage}'

    ANSWER:
    """
    start_time = time.time()
    gemini_result = gemini.send_message(message=metaprompt.format(query=quer, relevant_passage=results['documents'][0][0])).text
    end_time = time.time()
    total_end_time = time.time()
    display(Markdown(gemini_result))
    print(f"Gemini time: {end_time - start_time} seconds")
    print(f"Total time: {total_end_time - total_start_time} seconds")
    return gemini_result

In [9]:
for q in questions:
    print("--------------------------------------------------------------------")
    print(f"Question: {q}")
    gemini_result = generate_responses(q,collection)
    print("--------------------------------------------------------------------")

--------------------------------------------------------------------
Question: What is Instagram's current business proposal?
Document retrieved: business_docs_Business Proposal.md
Retrieval time: 0.4140923023223877 seconds


Instagram's business proposals are designed to help businesses enhance their social media presence, engage with their audience, and achieve their marketing goals. Here's a breakdown of each proposal:

**1. Social Media Marketing Services:**

* Instagram offers a range of social media marketing services to help businesses create and execute effective social media campaigns. 
* These services include strategic content creation, targeted ad campaigns, audience analysis, and performance tracking. 
* By partnering with Instagram's team of experienced digital marketers, businesses can maximize their reach and engagement on the platform.

**2. Influencer Collaboration Services:**

* Instagram connects brands with relevant influencers to drive brand awareness and engagement. 
* The platform's influencer collaboration services include influencer discovery, outreach, negotiation, campaign management, and performance analysis. 
* Businesses can leverage the reach and credibility of influencers to promote their products or services to a wider audience.

**3. Content Creation and Curation Services:**

* Instagram's creative team can help businesses elevate their visual presence and storytelling on the platform. 
* The content creation and curation services include bespoke content planning, photo and video production, and content curation. 
* By partnering with Instagram's experts, businesses can create engaging and visually appealing content that resonates with their target audience.

Gemini time: 5.986338138580322 seconds
Total time: 6.40043044090271 seconds
--------------------------------------------------------------------
--------------------------------------------------------------------
Question: Can you provide details about the marketing plan for Instagram?
Document retrieved: business_docs_Marketing Plan.md
Retrieval time: 0.41709327697753906 seconds


Sure, here is a summary of Instagram's marketing plan for their social media marketing services:

**Target Audience:**

* Small businesses, startups, and e-commerce ventures looking to enhance their online presence
* Brands seeking to engage with a younger demographic through social media platforms

**Marketing Objectives:**

* Increase brand awareness and reach through targeted social media campaigns
* Drive customer engagement and conversions for client businesses

**Strategies:**

* Develop personalized social media marketing strategies based on client goals and industry trends
* Utilize influencer collaborations, interactive content, and data-driven insights for optimized marketing campaigns

**Tactics:**

* Content creation and curation for social media platforms
* Paid advertising campaigns targeting specific audience segments
* Regular performance tracking and analytics to measure campaign effectiveness

**Budget:**

* Initial budget allocation of $20,000 for social media marketing campaigns and ad spend
* Budget may be adjusted based on campaign performance and client requirements

**Timeline:**

* Campaign implementation and monitoring over a 3-month period
* Monthly reporting and analysis to track performance metrics and adjust strategies

**Performance Metrics:**

* Key performance indicators include website traffic, engagement rates, lead generation, and conversion metrics
* Monthly reports to be generated for clients to review campaign progress and ROI

Gemini time: 6.091361045837402 seconds
Total time: 6.5094544887542725 seconds
--------------------------------------------------------------------
--------------------------------------------------------------------
Question: What information does the progress report of Instagram contain?
Document retrieved: business_docs_Progress Report.md
Retrieval time: 0.41509270668029785 seconds


...

Gemini time: 7.978782892227173 seconds
Total time: 8.39387559890747 seconds
--------------------------------------------------------------------
--------------------------------------------------------------------
Question: How much bonus does the Public Relations Manager receive?
Document retrieved: employee_payslips_employee_payslip_Mia Hernandez.md
Retrieval time: 0.4180929660797119 seconds


The Public Relations Manager receives a bonus of $1,000.00.

Gemini time: 7.589696168899536 seconds
Total time: 8.007789134979248 seconds
--------------------------------------------------------------------
--------------------------------------------------------------------
Question: What is the net pay to the employee working in the Customer Support department?
Document retrieved: employee_payslips_employee_payslip_William Anderson.md
Retrieval time: 0.3960883617401123 seconds


The net pay to the employee working in the Customer Support department is $3,275.00.

Gemini time: 8.721948862075806 seconds
Total time: 9.119037628173828 seconds
--------------------------------------------------------------------


In [6]:
questions = [
"How does Instagram's marketing plan contribute to its growth strategy?",
"In what ways do employee contracts at Instagram reflect the company's values?",
"What impact does diversity, inclusion, and equity have on Instagram's workplace culture?",
"How does Instagram's tax return history compare to industry standards?",
"What strategies does Instagram employ to ensure financial stability, as seen in its balance sheet?",
"How do the terms of the employment contract boost employee retention?",
"How does Instagram's marketing plan adapt to changing market trends and customer preferences?",
"What role does diversity, inclusion, and equity play in shaping Instagram's corporate culture?",
"How do Instagram's employee ESOPs align with the company's long-term growth objectives?",
"What impact does Instagram's tax return strategy have on its overall financial health?",
"How does Instagram's progress report reflect the company's performance compared to competitors?",
"What are the implications of Instagram's shareholder agreements on corporate governance?",
"How does Instagram's balance sheet reflect its investment and financial decision-making?",
"What are the challenges faced by Instagram's board of directors in maintaining transparency and accountability?",
"How does Instagram's marketing plan integrate digital and traditional marketing channels for maximum impact?"
]

In [7]:
for q in questions:
    print("--------------------------------------------------------------------")
    print(f"Question: {q}")
    gemini_result = generate_responses(q,collection)
    print("--------------------------------------------------------------------")

--------------------------------------------------------------------
Question: How does Instagram's marketing plan contribute to its growth strategy?
Document retrieved: business_docs_Marketing Plan.md
Retrieval time: 0.41709280014038086 seconds


Sure. Instagram's marketing plan plays a crucial role in the company's growth strategy by enabling it to reach new users, engage existing ones, and drive business results. Here's how:

**Target Audience Identification:** Instagram's marketing plan starts with identifying its target audience. The platform primarily focuses on individuals and businesses looking to connect, share, and grow their presence online. By understanding their audience's demographics, interests, and behaviors, Instagram can tailor its marketing efforts to resonate with them effectively.

**Content Strategy:** Instagram's marketing plan revolves around creating and distributing valuable content that meets the needs and interests of its target audience. The platform offers various content formats, including photos, videos, Stories, Reels, and IGTV, allowing businesses and individuals to showcase their products, services, or personal brand in engaging ways. By providing a platform for creative expression and storytelling, Instagram attracts and retains users, fostering a sense of community and belonging.

**Influencer Partnerships:** Influencer marketing is a key component of Instagram's growth strategy. The platform collaborates with influential individuals who have a large following and can help promote Instagram's features and services to their audiences. These partnerships allow Instagram to tap into new markets, increase brand awareness, and drive user engagement. By leveraging the credibility and reach of influencers, Instagram can effectively connect with target audiences and build trust.

**Community Engagement:** Instagram's marketing plan emphasizes building and engaging with its community. The platform encourages users to interact with each other through likes, comments, shares, and direct messages. Instagram also hosts contests, challenges, and live events to foster a sense of community and keep users engaged. By creating opportunities for interaction and connection, Instagram strengthens its user base and promotes brand loyalty.

**Data-Driven Insights:** Instagram's marketing plan is driven by data and insights. The platform collects and analyzes user data to understand their preferences, behaviors, and engagement patterns. This data helps Instagram refine its marketing strategies, optimize content, and tailor its services to better meet the needs of its users. By leveraging data-driven insights, Instagram can make informed decisions and maximize the effectiveness of its marketing efforts.

Overall, Instagram's marketing plan contributes to its growth strategy by attracting new users, engaging existing ones, and fostering a strong sense of community. The platform's focus on content, influencer partnerships, community engagement, and data-driven insights enables it to stay relevant, innovative, and competitive in the ever-evolving social media landscape.

Gemini time: 12.937891006469727 seconds
Total time: 13.354983806610107 seconds
--------------------------------------------------------------------
--------------------------------------------------------------------
Question: In what ways do employee contracts at Instagram reflect the company's values?
Document retrieved: hr_docs_filled_Employee Handbook.md
Retrieval time: 0.4090909957885742 seconds


Instagram's employee contracts reflect the company's values in several ways:

**Alignment with Mission and Values:** The contracts clearly state that employees are expected to align with Instagram's mission and values, which include creativity, diversity, and inclusion. This sets the expectation that employees will conduct themselves in a manner that supports and promotes these values in their work and interactions with colleagues and customers.

**Code of Conduct:** The contracts outline a Code of Conduct that employees must adhere to. This code includes guidelines for professional behavior, respect for others, confidentiality, and compliance with company policies and procedures. By incorporating these principles into the contracts, Instagram ensures that employees understand and are held accountable for maintaining a positive and ethical work environment.

**Anti-Discrimination and Harassment Policies:** The contracts emphasize Instagram's commitment to providing a work environment free from discrimination and harassment. They state that any form of discrimination or harassment based on race, gender, age, religion, or any other protected characteristic will not be tolerated. This demonstrates Instagram's commitment to fostering an inclusive and respectful workplace where all employees feel valued and supported.

**Performance Management:** The contracts outline the company's performance management processes, which are designed to provide regular feedback and support employee growth. By setting clear expectations and providing opportunities for professional development, Instagram shows that it values employee performance and is invested in their success.

**Employee Benefits:** The contracts describe the various employee benefits offered by Instagram, such as health insurance, retirement plans, and paid time off. These benefits demonstrate Instagram's commitment to employee well-being and work-life balance, reflecting the company's values of caring for its employees and creating a supportive work environment.

Overall, Instagram's employee contracts serve as a reflection of the company's values and commitment to creating a positive and inclusive workplace where employees are valued, supported, and empowered to succeed.

Gemini time: 8.82497239112854 seconds
Total time: 9.234063386917114 seconds
--------------------------------------------------------------------
--------------------------------------------------------------------
Question: What impact does diversity, inclusion, and equity have on Instagram's workplace culture?
Document retrieved: company_bylaws_Diversity, Equity, and Inclusion.md
Retrieval time: 0.41309237480163574 seconds


...

Gemini time: 5.394205570220947 seconds
Total time: 5.783292293548584 seconds
--------------------------------------------------------------------
--------------------------------------------------------------------
Question: What are the challenges faced by Instagram's board of directors in maintaining transparency and accountability?
Document retrieved: company_bylaws_Board of Directors.md
Retrieval time: 0.38608646392822266 seconds


**Challenges in Maintaining Transparency and Accountability:** Despite the commitment of Instagram's board of directors to transparency and accountability, there are several challenges they may face:

**Balancing Stakeholder Interests:** The board must navigate the sometimes competing interests of different stakeholders, including shareholders, employees, users, and the broader community. Striking a balance between maximizing shareholder value and fulfilling the company's social responsibilities can be a delicate task.

**Regulatory Scrutiny:** Instagram operates in a rapidly evolving regulatory landscape, with governments around the world implementing new laws and regulations related to social media, data privacy, and content moderation. The board must stay abreast of these regulations and ensure that Instagram complies with all applicable laws, which can be complex and challenging.

**Managing Public Perception:** As a highly visible social media platform, Instagram is constantly in the public eye. The board must manage Instagram's reputation and respond effectively to public criticism or controversies that may arise. Maintaining transparency and accountability is crucial for building and preserving public trust.

**Balancing Innovation and Safety:** Instagram must continually innovate and introduce new features to remain competitive and meet the evolving needs of its users. However, the board must also ensure that these innovations do not compromise user safety or privacy. Striking the right balance between innovation and safety is an ongoing challenge for the board.

Despite these challenges, Instagram's board of directors is committed to maintaining high standards of transparency and accountability. By engaging with stakeholders, responding to feedback, and adhering to ethical and legal guidelines, the board aims to foster a culture of trust and ensure that Instagram operates in a responsible and sustainable manner.

Gemini time: 10.360315084457397 seconds
Total time: 10.74640154838562 seconds
--------------------------------------------------------------------
--------------------------------------------------------------------
Question: How does Instagram's marketing plan integrate digital and traditional marketing channels for maximum impact?
Document retrieved: business_docs_Marketing Plan.md
Retrieval time: 0.40609097480773926 seconds


Instagram's marketing plan integrates digital and traditional marketing channels to maximize its impact in several ways:

**Digital Marketing:** Instagram leverages its social media platform as a primary digital marketing channel. It utilizes targeted advertising, influencer partnerships, and content creation to reach its vast user base. By creating engaging and shareable content, Instagram attracts and retains users, driving brand awareness and engagement.

**Traditional Marketing:** While Instagram primarily focuses on digital marketing, it also incorporates traditional marketing channels to complement its digital efforts. For example, Instagram has been known to use print advertising, billboards, and TV commercials to promote its platform and specific campaigns. These traditional channels help Instagram reach a broader audience and build brand recognition beyond its digital presence.

**Integration and Synergy:** Instagram's marketing plan effectively integrates digital and traditional channels to create a synergistic effect. By using both digital and traditional channels, Instagram can reach a wider audience and reinforce its messaging across multiple platforms. This multi-channel approach helps Instagram maximize its impact, increase brand visibility, and drive engagement.

**Targeted Advertising:** Instagram's targeted advertising capabilities allow it to precisely reach specific audience segments based on their demographics, interests, and behaviors. This enables Instagram to deliver highly relevant ads to users, increasing the likelihood of engagement and conversions.

**Influencer Collaborations:** Instagram's influencer marketing program connects brands with influential individuals who have a dedicated following on the platform. These collaborations allow Instagram to tap into the credibility and reach of influencers, helping brands build trust and authenticity with their target audience.

**Content Creation:** Instagram's focus on content creation is central to its marketing strategy. The platform encourages users to share photos, videos, and stories, creating a vast pool of user-generated content. Instagram also produces original content, such as IGTV shows and Reels, to entertain and engage its users.

Overall, Instagram's marketing plan is a well-rounded integration of digital and traditional marketing channels that maximizes its impact, builds brand awareness, drives engagement, and supports its overall growth strategy.

Gemini time: 13.435001373291016 seconds
Total time: 13.841092348098755 seconds
--------------------------------------------------------------------


In [8]:
question_multihop = [
"Given Instagram's progress report data, what are the key factors influencing its marketing strategy?",
"How do Instagram's shareholder agreements align with the company's long-term goals and vision?",
"Considering Instagram's board of directors and diversity initiatives, how are decisions made to promote inclusion within the company?",
"By analyzing Instagram's income statement and balance sheet, what insights can be drawn regarding its financial performance and future projections?",
"Exploring Instagram's employee ESOPs and payslip details, how does the company incentivize employee retention and satisfaction?",
"By analyzing Instagram's progress report and employee contracts, what insights can be drawn regarding the company's growth trajectory and workforce management strategies?",
"Considering Instagram's marketing plan, shareholder agreements, and balance sheet data, how does the company plan to expand its market share while maintaining financial stability?",
"How do Instagram's employee ESOPs, diversity initiatives, and progress report metrics contribute to its reputation as an employer of choice?",
"Given Instagram's tax return history, shareholder agreements, and board of directors' decisions, what strategies does the company employ to manage financial risks and compliance?",
"How do Instagram's marketing strategies, employee payslip structures, and balance sheet figures align with its mission and values?",
"Exploring Instagram's progress report, business proposal, and diversity initiatives, what are the key drivers of the company's competitive advantage?",
"By examining Instagram's marketing plan, employee contracts, and income statement details, how does the company ensure ethical marketing practices and financial transparency?",
"What insights can be gained from analyzing Instagram's shareholder agreements, progress report metrics, and employee ESOPs regarding the company's commitment to sustainability and social responsibility?",
"Considering Instagram's tax return data, board of directors' decisions, and employee payslip information, how does the company foster a culture of innovation and continuous improvement?",
"Based on Instagram's marketing strategies, balance sheet trends, and diversity metrics, what recommendations can be made to enhance the company's overall performance and stakeholder value?"
]

In [9]:
for q in question_multihop:
    print("--------------------------------------------------------------------")
    print(f"Question: {q}")
    gemini_result = generate_responses(q,collection)
    print("--------------------------------------------------------------------")

--------------------------------------------------------------------
Question: Given Instagram's progress report data, what are the key factors influencing its marketing strategy?
Document retrieved: business_docs_Progress Report.md
Retrieval time: 1.6163609027862549 seconds


The provided progress report does not contain specific information about Instagram's marketing strategy or the factors influencing it. Therefore, I cannot answer this question from the provided context.

Gemini time: 4.551016807556152 seconds
Total time: 6.167377710342407 seconds
--------------------------------------------------------------------
--------------------------------------------------------------------
Question: How do Instagram's shareholder agreements align with the company's long-term goals and vision?
Document retrieved: company_bylaws_Shareholders.md
Retrieval time: 0.4120924472808838 seconds


Instagram's shareholder agreements align with the company's long-term goals and vision by establishing a clear ownership structure and aligning the interests of shareholders with the company's success. Here's how:

**Control and Decision-Making:** As the sole owner of Instagram, Facebook Inc. has complete control over the company's strategic direction, decision-making, and resource allocation. This centralized control allows Facebook to ensure that Instagram's long-term goals and vision are aligned with its own overall strategy.

**Alignment of Interests:** The shareholdings of major shareholders, such as the CEO, Chairman of the Board, and CFO, are significant. This alignment of interests incentivizes these key individuals to make decisions that benefit the company's long-term growth and profitability. When the interests of shareholders are aligned with the company's success, it reduces the risk of short-term or self-serving decisions that could jeopardize Instagram's long-term goals.

**Financial Stability and Growth:** The financial resources and support provided by Facebook Inc. contribute to Instagram's financial stability and growth. Facebook's investment in Instagram allows the company to invest in new features, expand its user base, and pursue strategic acquisitions. This financial backing supports Instagram's long-term vision of becoming a leading player in the social media and visual storytelling space.

**Innovation and Adaptability:** Shareholder agreements can provide a framework for innovation and adaptability. By allowing Facebook Inc. to have ultimate control over Instagram's direction, the company can quickly respond to changing market trends, technological advancements, and competitive pressures. This flexibility enables Instagram to remain innovative and adapt to the evolving needs of its users and the industry.

Overall, Instagram's shareholder agreements are designed to align the interests of shareholders with the company's long-term goals and vision. By establishing clear ownership structures, incentivizing key individuals, providing financial stability, and fostering innovation, these agreements contribute to Instagram's sustainable growth and success in the competitive social media landscape.

Gemini time: 14.718289136886597 seconds
Total time: 15.131381511688232 seconds
--------------------------------------------------------------------
--------------------------------------------------------------------
Question: Considering Instagram's board of directors and diversity initiatives, how are decisions made to promote inclusion within the company?
Document retrieved: company_bylaws_Diversity, Equity, and Inclusion.md
Retrieval time: 0.3900871276855469 seconds


...

Gemini time: 9.20505690574646 seconds
Total time: 9.607146501541138 seconds
--------------------------------------------------------------------
--------------------------------------------------------------------
Question: Considering Instagram's tax return data, board of directors' decisions, and employee payslip information, how does the company foster a culture of innovation and continuous improvement?
Document retrieved: hr_docs_filled_Employee Handbook.md
Retrieval time: 0.40409016609191895 seconds


I am unable to provide insights into Instagram's culture of innovation and continuous improvement as the provided text does not include tax return data, board of directors' decisions, or employee payslip information.

Gemini time: 9.002514362335205 seconds
Total time: 9.406604528427124 seconds
--------------------------------------------------------------------
--------------------------------------------------------------------
Question: Based on Instagram's marketing strategies, balance sheet trends, and diversity metrics, what recommendations can be made to enhance the company's overall performance and stakeholder value?
Document retrieved: business_docs_Marketing Plan.md
Retrieval time: 0.4090914726257324 seconds


I am unable to provide recommendations to enhance Instagram's overall performance and stakeholder value as the provided text does not include balance sheet trends or diversity metrics.

Gemini time: 9.140545845031738 seconds
Total time: 9.54963731765747 seconds
--------------------------------------------------------------------
